In [1]:
import pandas as pd

In [2]:
boxscores = pd.read_csv("/Users/noord/Documents/Work/Classes/csc466/final/nba_regressor/data/archive/PlayerStatistics.csv")

/var/folders/sx/gnb_mn3n55d3fg_tbqfpx6dw0000gn/T/ipykernel_46655/3247145265.py:1: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  boxscores = pd.read_csv("/Users/noord/Documents/Work/Classes/csc466/final/nba_regressor/data/archive/PlayerStatistics.csv")


In [ ]:
# lamelo ball
lamelo = boxscores[boxscores["personId"] == 1630163]

print(lamelo.head())

     firstName lastName  personId    gameId              gameDate  \
2122    LaMelo     Ball   1630163  22500143  2025-11-01T18:00:00Z   
2472    LaMelo     Ball   1630163  22500138  2025-10-30T19:00:00Z   
2841    LaMelo     Ball   1630163  22500124  2025-10-28T19:30:00Z   
3314    LaMelo     Ball   1630163  22500109  2025-10-26T18:00:00Z   
3486    LaMelo     Ball   1630163  22500102  2025-10-25T19:30:00Z   

     playerteamCity playerteamName opponentteamCity opponentteamName gameType  \
2122      Charlotte        Hornets        Minnesota     Timberwolves      NaN   
2472      Charlotte        Hornets          Orlando            Magic      NaN   
2841      Charlotte        Hornets            Miami             Heat      NaN   
3314      Charlotte        Hornets       Washington          Wizards      NaN   
3486      Charlotte        Hornets     Philadelphia            76ers      NaN   

      ... threePointersPercentage freeThrowsAttempted  freeThrowsMade  \
2122  ...                

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import pandas as pd

file_path = '/content/gdrive/My Drive/csc-466-nba-regressor/data/archive/PlayerStatistics.csv'
boxscores = pd.read_csv(file_path)
print(boxscores.head())

games = pd.read_csv('/contant/gdrive/My Drive/csc-466-nba-regressor/data/archive/')

In [ ]:
# lamelo ball
lamelo = boxscores[boxscores["personId"] == 1630163]

print(lamelo.head())

In [ ]:
print(boxscores.columns)

In [ ]:
!pip install nba_api

In [ ]:
from nba_api.stats.endpoints import playergamelog
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

In [ ]:
giannis_id = 203507 # giannis
seasons = [f"{y-1}-{str(y)[-2:]}" for y in range(2025, 2020, -1)]
all_games = []

In [ ]:
for season in seasons:
    gamelog = playergamelog.PlayerGameLog(player_id=giannis_id, season=season)
    df = gamelog.get_data_frames()[0]
    df['SEASON'] = season
    all_games.append(df[['GAME_DATE', 'MATCHUP', 'PTS', 'SEASON']])

giannis = pd.concat(all_games, ignore_index=True)
giannis['GAME_DATE'] = pd.to_datetime(giannis['GAME_DATE'])

In [ ]:
giannis['OPPONENT'] = (
    giannis['MATCHUP']
    .str.extract(r'@ ([A-Z]{2,3})|vs ([A-Z]{2,3})')
    .bfill(axis=1)
    .iloc[:, 0]
)
giannis = giannis.sort_values('GAME_DATE')

In [ ]:
avg_vs_team = giannis.groupby('OPPONENT')['PTS'].mean().sort_values(ascending=False)

plt.figure(figsize=(10,5))
sns.barplot(x=avg_vs_team.index, y=avg_vs_team.values, palette='coolwarm')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
for season in seasons:
    gamelog = playergamelog.PlayerGameLog(player_id=giannis_id, season=season)
    df = gamelog.get_data_frames()[0]
    df['SEASON'] = season
    df['TEAM_ABBREVIATION'] = 'MIL'
    all_games.append(df[['GAME_DATE', 'MATCHUP', 'PTS', 'SEASON', 'TEAM_ABBREVIATION']])
giannis = pd.concat(all_games, ignore_index=True)
giannis['GAME_DATE'] = pd.to_datetime(giannis['GAME_DATE'])

In [ ]:
def parse_matchup(matchup):
    if ' @ ' in matchup or '@ ' in matchup or ' @' in matchup:
        opponent = matchup.split('@')[1].strip().replace('.', '').split()[0]
        is_home = 0
    elif ' vs. ' in matchup or ' vs ' in matchup:
        parts = matchup.split('vs')[-1].strip().replace('.', '').split()
        opponent = parts[0] if parts else matchup.split()[-1]
        is_home = 1
    else:
        opponent = matchup.strip().split()[-1].replace('.', '')
        is_home = 1 if 'vs' in matchup.lower() else 0

    return pd.Series({'OPPONENT': opponent, 'IS_HOME': is_home})

giannis[['OPPONENT', 'IS_HOME']] = giannis['MATCHUP'].apply(parse_matchup)
giannis['LOCATION'] = giannis['IS_HOME'].map({1: 'Home', 0: 'Away'})


In [ ]:
avg_home_away = (
    giannis.groupby(['OPPONENT', 'LOCATION'])['PTS']
    .mean()
    .reset_index()
    .pivot(index='OPPONENT', columns='LOCATION', values='PTS')
    .fillna(0)
)

for col in ['Home', 'Away']:
    if col not in avg_home_away.columns:
        avg_home_away[col] = 0

avg_home_away['Diff (Home - Away)'] = avg_home_away['Home'] - avg_home_away['Away']
avg_home_away = avg_home_away.sort_values('Home', ascending=False)

avg_home_away

In [ ]:
avg_home_away[['Home', 'Away']].plot(kind='bar', figsize=(12,6), color=['blue', 'orange'])
plt.title("Giannis Antetokounmpo - Avg Points vs Each Opponent (Home vs Away, Last 5 Seasons)")
plt.xlabel("Opponent")
plt.ylabel("Average Points")
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.legend(title="Location")
plt.tight_layout()
plt.show()